In [1]:
import pickle
import torch
import os
import pandas as pd
from tqdm import tqdm
import re
import json

tqdm.pandas()

import sys
sys.path.insert(0,'../../')
from caches.ner_to_sentence_insertion import NerToSentenceInsertion

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-26 17:56:22.052353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 17:56:22.248917: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-26 17:56:23.084510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file

In [3]:
ner = NerToSentenceInsertion(model_path='../../../model-best/')

In [26]:
test_df = pd.read_csv(
    '../wikidata-simplequestions/annotated_wd_data_test_answerable.txt',
    sep='\t',
    names=['S', 'P', 'O', 'Q'],
    # nrows=200,
)
test_df.head()

,S,P,O,Q
0,Q7358590,P20,Q1637790,Where did roger marquis die
1,Q154335,P509,Q12152,what was the cause of death of yves klein
2,Q2747238,P413,Q5059480,What position does carlos gomez play?
3,Q62498,P21,Q6581097,how does engelbert zaschka identify
4,Q182485,P413,Q1143358,what position does pee wee reese play in baseball


In [27]:
test_df['Q_with_NER'] = test_df['Q'].progress_apply(ner.entity_labeling)
test_df.head()

100%|██████████| 5622/5622 [00:00<00:00, 243896.25it/s]


,S,P,O,Q,Q_with_NER
0,Q7358590,P20,Q1637790,Where did roger marquis die,Where Did [START] Roger Marquis [END] Die
1,Q154335,P509,Q12152,what was the cause of death of yves klein,What Was The Cause Of Death Of [START] Yves Kl...
2,Q2747238,P413,Q5059480,What position does carlos gomez play?,What Position Does [START] Carlos Gomez [END] ...
3,Q62498,P21,Q6581097,how does engelbert zaschka identify,How Does [START] Engelbert Zaschka [END] Identify
4,Q182485,P413,Q1143358,what position does pee wee reese play in baseball,What Position Does [START] Pee Wee Reese [END]...


In [5]:
test_df.to_csv('./WD_SQ_with_NER.csv', index=False)

In [4]:
!git clone https://github.com/vladislavneon/RuBQ.git

Cloning into 'RuBQ'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 106 (delta 56), reused 29 (delta 15), pack-reused 0
Receiving objects: 100% (106/106), 14.13 MiB | 6.46 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [5]:
with open('./RuBQ/RuBQ_2.0/RuBQ_2.0_test.json', 'r') as f:
    rubq_test = json.load(f)

In [31]:
res = []
for row in tqdm(rubq_test):
    try:
        q_entity = re.findall(r'wd\:Q[0-9]+', row['query'])[0].split(':')[1]
        question_with_ner = ner.entity_labeling(row['question_eng'])
        res.append({
            'S': q_entity,
            'Q': row['question_eng'],
            'Q_with_NER': question_with_ner,
        })
    except:
        pass


100%|██████████| 2330/2330 [00:51<00:00, 44.94it/s] 


In [34]:
pd.DataFrame(res).to_csv('RuBQ_test_part4ner.csv', index=False)